In [38]:

import yaml
import pandas as pd
import requests
import io
import os

# create folders to save data (git does not track empty directories)

os.makedirs('data/', exist_ok=True)
os.makedirs('data/processed/', exist_ok=True)  
os.makedirs('data/raw/', exist_ok=True)  

# load login credentials

with open("config/login.yaml", "r", encoding="utf-8") as file:
    conf = yaml.safe_load(file)

config = {
    "path": conf["access_path"],
    "format": conf["format"]
}

login = {
    "username": conf["username"],
    "password": conf["password"]
}


## Microcensus data

The microcensus is a survey conducted by the national statistical office that collects data about a fraction of the total national population. Let us look at which data tables exist for the microcensus:

In [2]:
def get_table_list(selection, parameters, config):
    parameters["selection"] = selection
    url = f"{config['path']}/catalogue/tables"
    return requests.get(url, params=parameters).json()

In [3]:
relevant_tables = get_table_list("12211*", login, config)
relevant_tables

{'Ident': {'Service': 'catalogue', 'Method': 'tables'},
 'Status': {'Code': 0, 'Content': 'erfolgreich', 'Type': 'Information'},
 'Parameter': {'username': '********************',
  'password': '********************',
  'selection': '12211*',
  'area': 'Öffentlich',
  'searchcriterion': 'Code',
  'sortcriterion': 'Code',
  'pagelength': '100',
  'language': 'de'},
 'List': [{'Code': '12211-0001',
   'Content': 'Bevölkerung, Erwerbstätige, Erwerbslose, Erwerbspersonen, Nichterwerbspersonen aus Hauptwohnsitzhaushalten: Deutschland, Jahre, Geschlecht, Altersgruppen',
   'Time': '2020 - 2022'},
  {'Code': '12211-0002',
   'Content': 'Bevölkerung, Erwerbstätige, Erwerbslose, Erwerbspersonen, Nichterwerbspersonen aus Hauptwohnsitzhaushalten: Deutschland, Jahre, Überwiegender Lebensunterhalt, Altersgruppen',
   'Time': '2020 - 2022'},
  {'Code': '12211-0003',
   'Content': 'Bevölkerung, Erwerbstätige, Erwerbslose, Erwerbspersonen,\nNichterwerbspersonen aus Hauptwohnsitzhaush.: Deutschland,\nJ

Let us look at the data for a particular table, e.g. `12211-0003` looks potentially interesting because it has info on income. 

In [50]:

def get_data(table, parameters, config):
    csv_path = f"data/raw/tablefile_{table}.csv"
    if os.path.isfile(csv_path):
            data = pd.read_csv(csv_path)
    else:
            data = get_data_from_api(table, parameters, config)
    return data

        
def get_data_from_api(table, parameters, config):
    print(f"Requesting table data for table '{table}'")
    parameters["name"] = table

    # use "ffcsv" format because it is consistent across tables
    # and over time
    parameters["format"] = config["format"]
    url = f"{config['path']}/data/tablefile"
    
    response = requests.get(url, params=parameters)
    if response.ok:
        data = pd.read_csv(
            io.StringIO(response.content.decode("utf-8")),
            sep = ";",
            decimal = ",",
            na_values = ["-"],
        )
        data.to_csv(f"data/raw/tablefile_{table}.csv", index=False)
        return data
    else:
        return response


In [55]:
my_table = "12211-0003"

data = get_data(my_table, login, config)


In [56]:
data.head()

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,...,2_Auspraegung_Label,3_Merkmal_Code,3_Merkmal_Label,3_Auspraegung_Code,3_Auspraegung_Label,BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000,ERW041__Erwerbstaetige_aus_Hauptwohnsitzhaushalten__1000,ERW042__Erwerbslose_aus_Hauptwohnsitzhaushalten__1000,ERW040__Erwerbspersonen_aus_Hauptwohnsitzhaushalten__1000,ERW043__Nichterwerbspersonen_aus_Hauptwohnsitzhaushalten__1000
0,12211,Mikrozensus,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,GES,...,männlich,EKMGK2,Größenklassen des persönl. monatl. Nettoeinkom...,EUR00000B00000,Kein Einkommen,6048,66,94,161,5887
1,12211,Mikrozensus,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,GES,...,männlich,EKMGK2,Größenklassen des persönl. monatl. Nettoeinkom...,EUR00000B00500,unter 500 EUR,3279,653,161,813,2466
2,12211,Mikrozensus,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,GES,...,männlich,EKMGK2,Größenklassen des persönl. monatl. Nettoeinkom...,EUR00500B01000,500 bis unter 1000 EUR,3579,1469,255,1724,1854
3,12211,Mikrozensus,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,GES,...,männlich,EKMGK2,Größenklassen des persönl. monatl. Nettoeinkom...,EUR01000B01250,1000 bis unter 1250 EUR,2356,957,66,1023,1333
4,12211,Mikrozensus,JAHR,Jahr,2022,DINSG,Deutschland insgesamt,DG,Deutschland,GES,...,männlich,EKMGK2,Größenklassen des persönl. monatl. Nettoeinkom...,EUR01250B01500,1250 bis unter 1500 EUR,2515,1255,48,1304,1212


In [35]:
my_data.columns

Index(['Statistik_Code', 'Statistik_Label', 'Zeit_Code', 'Zeit_Label', 'Zeit',
       '1_Merkmal_Code', '1_Merkmal_Label', '1_Auspraegung_Code',
       '1_Auspraegung_Label', '2_Merkmal_Code', '2_Merkmal_Label',
       '2_Auspraegung_Code', '2_Auspraegung_Label', '3_Merkmal_Code',
       '3_Merkmal_Label', '3_Auspraegung_Code', '3_Auspraegung_Label',
       'BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000',
       'ERW041__Erwerbstaetige_aus_Hauptwohnsitzhaushalten__1000',
       'ERW042__Erwerbslose_aus_Hauptwohnsitzhaushalten__1000',
       'ERW040__Erwerbspersonen_aus_Hauptwohnsitzhaushalten__1000',
       'ERW043__Nichterwerbspersonen_aus_Hauptwohnsitzhaushalten__1000'],
      dtype='object')